In [12]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\Business-Chatbot\\research'

In [13]:
import os
os.chdir('../')

In [14]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\Business-Chatbot'

In [15]:
%pip install langchain
%pip install -U langchain-community

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
# Extracting text from PDF files in a directory
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
          glob="*/*.pdf", 
          loader_cls=PyPDFLoader
          )
    
    documents = loader.load()
   
    return documents

In [17]:
%pip install pypdf

from langchain.document_loaders import PyPDFLoader

# Path to your PDF
pdf_path = r"C:\Users\hp\OneDrive\Desktop\Business-Chatbot\data\ecommerce_faq_extended.pdf"

# Load PDF
loader = PyPDFLoader(pdf_path)
extracted_data = loader.load()

# Show results
print(len(extracted_data))   # Number of chunks
print(extracted_data[0])     # First chunk


Note: you may need to restart the kernel to use updated packages.
5
page_content='■ Orders & Delivery
Q: How can I track my order?
A: You can track your order in the 'My Orders' section of your account. Click on the specific order to
see live tracking updates.
Q: My order is delayed, what should I do?
A: Please check the tracking details first. If your order is delayed beyond the expected date, contact
customer support for assistance.
Q: Can I change my delivery address after placing an order?
A: Yes, you can change your delivery address before the item is shipped. Go to 'My Orders' → 'Edit
Address'.
Q: Can I schedule delivery for a specific time?
A: Currently, scheduled delivery is available in selected cities only. You can choose the delivery slot
during checkout if eligible.
■ Returns, Refunds & Cancellations
Q: How do I return a product?
A: Go to 'My Orders', select the item you want to return, and click 'Return Item'. Follow the
instructions to schedule pickup.
Q: What is the retu

In [18]:
extracted_data


[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-16T06:50:37+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-16T06:50:37+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'C:\\Users\\hp\\OneDrive\\Desktop\\Business-Chatbot\\data\\ecommerce_faq_extended.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content="■ Orders & Delivery\nQ: How can I track my order?\nA: You can track your order in the 'My Orders' section of your account. Click on the specific order to\nsee live tracking updates.\nQ: My order is delayed, what should I do?\nA: Please check the tracking details first. If your order is delayed beyond the expected date, contact\ncustomer support for assistance.\nQ: Can I change my delivery address after placing an order?\nA: Yes, you can change your delivery address before the item is shipped. Go to 'My Orders' → 'Edit\nAddress'.\

In [19]:
len(extracted_data)

5

In [20]:
from typing import List
from langchain.schema import Document  # or wherever your Document comes from

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs = []   # ✅ Correct initialization
    
    for doc in docs:
        src = doc.metadata.get("source")  # ✅ use `doc`, not `docs`
        page = doc.metadata.get("page")
        
        minimal_doc = Document(
            page_content=doc.page_content,
            metadata={"source": src, "page": page}
        )
        
        minimal_docs.append(minimal_doc)
    
    return minimal_docs


In [21]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [22]:
minimal_docs

[Document(metadata={'source': 'C:\\Users\\hp\\OneDrive\\Desktop\\Business-Chatbot\\data\\ecommerce_faq_extended.pdf', 'page': 0}, page_content="■ Orders & Delivery\nQ: How can I track my order?\nA: You can track your order in the 'My Orders' section of your account. Click on the specific order to\nsee live tracking updates.\nQ: My order is delayed, what should I do?\nA: Please check the tracking details first. If your order is delayed beyond the expected date, contact\ncustomer support for assistance.\nQ: Can I change my delivery address after placing an order?\nA: Yes, you can change your delivery address before the item is shipped. Go to 'My Orders' → 'Edit\nAddress'.\nQ: Can I schedule delivery for a specific time?\nA: Currently, scheduled delivery is available in selected cities only. You can choose the delivery slot\nduring checkout if eligible.\n■ Returns, Refunds & Cancellations\nQ: How do I return a product?\nA: Go to 'My Orders', select the item you want to return, and click '

In [23]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
    )
    
    text_chunks = text_splitter.split_documents(minimal_docs)
    return text_chunks


In [24]:
# 3. Split into smaller chunks using already loaded extracted_data
def text_split(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,    # length of each chunk
        chunk_overlap=50,  # overlap between chunks
    )
    return text_splitter.split_documents(docs)

text_chunks = text_split(extracted_data)

# 4. Print results
print(f"📄 Total Chunks Created: {len(text_chunks)}")
print("🔹 First Chunk Example:")
print(text_chunks[0])


📄 Total Chunks Created: 22
🔹 First Chunk Example:
page_content='■ Orders & Delivery
Q: How can I track my order?
A: You can track your order in the 'My Orders' section of your account. Click on the specific order to
see live tracking updates.
Q: My order is delayed, what should I do?
A: Please check the tracking details first. If your order is delayed beyond the expected date, contact
customer support for assistance.
Q: Can I change my delivery address after placing an order?' metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-16T06:50:37+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-16T06:50:37+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'C:\\Users\\hp\\OneDrive\\Desktop\\Business-Chatbot\\data\\ecommerce_faq_extended.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}


In [25]:
if 'text_chunks' not in globals():
	text_chunks = text_split(minimal_docs)
text_chunks

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-16T06:50:37+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-16T06:50:37+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'C:\\Users\\hp\\OneDrive\\Desktop\\Business-Chatbot\\data\\ecommerce_faq_extended.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content="■ Orders & Delivery\nQ: How can I track my order?\nA: You can track your order in the 'My Orders' section of your account. Click on the specific order to\nsee live tracking updates.\nQ: My order is delayed, what should I do?\nA: Please check the tracking details first. If your order is delayed beyond the expected date, contact\ncustomer support for assistance.\nQ: Can I change my delivery address after placing an order?"),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'cr

In [26]:
%pip install sentence-transformers

from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
     ) # Use "cuda" if you have a compatible GPU)
    return embeddings

embeddings = download_embeddings()

Note: you may need to restart the kernel to use updated packages.


C:\Users\hp\AppData\Local\Temp\ipykernel_10912\1412973440.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [27]:
vector  = embeddings.embed_query("Hello world")
vector

[-0.03447727859020233,
 0.031023195013403893,
 0.0067350007593631744,
 0.02610897831618786,
 -0.03936203941702843,
 -0.16030246019363403,
 0.06692393869161606,
 -0.006441489793360233,
 -0.04745043069124222,
 0.014758850447833538,
 0.07087530940771103,
 0.05552756413817406,
 0.019193332642316818,
 -0.026251299306750298,
 -0.010109570808708668,
 -0.026940522715449333,
 0.022307388484477997,
 -0.022226659581065178,
 -0.1496925801038742,
 -0.017493056133389473,
 0.00767620699480176,
 0.05435231328010559,
 0.003254441311582923,
 0.03172589838504791,
 -0.08462132513523102,
 -0.02940601296722889,
 0.0515955425798893,
 0.048124007880687714,
 -0.003314810572192073,
 -0.05827917903661728,
 0.04196931794285774,
 0.022210605442523956,
 0.12818878889083862,
 -0.022338947281241417,
 -0.011656264774501324,
 0.06292837113142014,
 -0.03287631645798683,
 -0.09122605621814728,
 -0.031175334006547928,
 0.052699584513902664,
 0.04703482240438461,
 -0.0842030793428421,
 -0.030056186020374298,
 -0.0207448396

In [28]:
print("vector Embedding:", len(vector))  # Dimension of the embedding vector

vector Embedding: 384


In [29]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [30]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
OPENROUTER_API_KEY=os.getenv("OPENROUTER_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

In [46]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone
pc = Pinecone(api_key="pcsk_5fVU4q_J7rcEoJwsDqKT9zQ8BW4maF74XhwapLyVCMEkzzj3KxFEAqTQb32kaDxv24DWnG")

index_name = "business-helper"


# Create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of embeddings (e.g., sentence-transformers all-MiniLM-L6-v2 = 384)
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

# ✅ Correct way to connect to the index
index = pc.Index(index_name)

print("Connected to index:", index_name)


ImportError: cannot import name 'Pinecone' from 'pinecone' (unknown location)

In [ ]:
import pinecone

# Initialize Pinecone
pinecone.init(
    api_key="pcsk_5fVU4q_J7rcEoJwsDqKT9zQ8BW4maF74XhwapLyVCMEkzzj3KxFEAqTQb32kaDxv24DWnG",       # replace with your API key
    environment="us-east1-gcp"    # check your Pinecone environment
)

index_name = "your-index"

# Create index if it does not exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, dimension=1536)  # embedding dimension


AttributeError: module 'pinecone' has no attribute 'init'

In [ ]:
from pinecone import Pinecone
pinecone_api_key=PINECONE_API_KEY
pc = Pinecone(api_key= pinecone_api_key)